<a href="https://colab.research.google.com/github/G-Gaddu/Quant-Material/blob/main/Market_Making_for_currencies.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [34]:
# Import the necessary packages
import yfinance as yf
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import random

In [35]:
import warnings
warnings.filterwarnings('ignore')


In [56]:
# Create a list of tickers for currencies
currencies = ['EURUSD=X', 'EURUSD=X', 'GBPJPY=X', 'JPY=X', 'GBP=X', 'GBPAUD=X', 'GBPCAD=X', 'GBPCAD=X','GBPCHF=X','GBPCNY=X','GBPINR=X','GBPNOK=X','GBPQAR=X','GBPZAR=X','EURCHF=X','EURJPY=X','EURSEK=X','EURCHF=X','EURHUF=X','CNY=X','HKD=X','SGD=X','INR=X','MXN=X','PHP=X','IDR=X','THB=X','MYR=X','ZAR=X','RUB=X' ]

In [37]:
# Create a function that will full the adj close prices for the currencies
def download_currency_data(currency, start_date, end_date):
    data = yf.download(currency, start=start_date, end=end_date)
    return data['Adj Close']

In [106]:
# Simulated market environment
class MarketMaker:
  def __init__(self, initial_investment, max_inventory, trading_amount, trading_cost=0.0025):
    self.inventory = 0  # Starting inventory is zero
    self.max_inventory = max_inventory
    self.trading_amount = trading_amount
    self.trading_cost = trading_cost  # Trading cost as a fraction (0.25% = 0.0025)
    self.cash = initial_investment
    self.total_trades = 0
    self.profit = 0
    self.initial_investment = initial_investment
    self.inventory_history = []
    self.cash_history = []
    self.profit_history = []

  def simulate_market(self, price_data):
    # Iterate through price data, simulating trades with dynamic spread
    for price in price_data:
      # Generate a random spread between 0.1% and 1%
      spread_percentage = random.uniform(0.001, 0.01)

      # Calculate the spread dynamically as a percentage of the transaction value
      spread = price * spread_percentage

      # Bid and Ask Prices
      bid_price = price - spread / 2
      ask_price = price + spread / 2

      # Simulate a trade based on inventory levels
      self.execute_order(bid_price, ask_price)

      # Record inventory, cash, and profit levels
      self.inventory_history.append(self.inventory)
      self.cash_history.append(self.cash)
      self.profit_history.append(self.profit)

  def execute_order(self, bid_price, ask_price):
    # Simulate order execution with trading costs
    # Buying at the bid price (if inventory allows)
    if self.inventory < self.max_inventory:
      transaction_value = bid_price * self.trading_amount
      trading_fee = transaction_value * self.trading_cost  # Apply trading cost on the transaction value
      self.inventory += self.trading_amount
      self.cash -= (transaction_value + trading_fee)  # Deduct the cost plus the fee
      self.total_trades += 1

      # Selling at the ask price (if inventory is above minimum)
      if self.inventory > -self.max_inventory:
        transaction_value = ask_price * self.trading_amount
        trading_fee = transaction_value * self.trading_cost  # Apply trading cost on the transaction value
        self.inventory -= self.trading_amount
        self.cash += (transaction_value - trading_fee)  # Add the revenue minus the fee
        self.total_trades += 1
        profit_per_trade = (ask_price - bid_price) * self.trading_amount - 2 * trading_fee  # Net profit after fees
        self.profit += profit_per_trade  # Calculate profit accounting for fees

  def get_performance_metrics(self):
    # Calculate profit as a percentage of the initial investment
    profit_percentage = (self.profit / self.initial_investment) * 100
    return {'Final Inventory': self.inventory,'Final Cash': self.cash,'Total Trades': self.total_trades,'Profit': self.profit,'Profit Percentage': profit_percentage}

  def plot_inventory_cash(self):
    # Plot inventory, cash, and profit levels over time
    plt.figure(figsize=(12, 8))

    plt.subplot(3, 1, 1)
    plt.plot(self.inventory_history, label='Inventory')
    plt.title('Inventory over Time')
    plt.legend()

    plt.subplot(3, 1, 2)
    plt.plot(self.cash_history, label='Cash', color='orange')
    plt.title('Cash over Time')
    plt.legend()

    plt.subplot(3, 1, 3)
    plt.plot(self.profit_history, label='Profit', color='green')
    plt.title('Profit over Time')
    plt.legend()

    plt.tight_layout()
    plt.show()



In [112]:
# Parameters for Market Making
currency_tickers = currencies
start_date = '2023-01-01'
end_date = '2023-12-31'
initial_investment = 1000000  # Initial investment of $1 million per currency
max_inventory = 1000  # Maximum units to hold in inventory
trading_amount = 100  # Amount of currency to trade per transaction
trading_cost = 0.0025  # 0.25% trading cost per transaction
num_simulations = 10

In [113]:
# Download data and simulate for multiple currencies
performance_results = []  # Initialize a list to store performance metrics for all currencies

for currency in currency_tickers:
  # Download price data for each currency
  price_data = download_currency_data(currency, start_date, end_date)
  price_data = price_data.values  # Convert to array for easier processing

  # Initialize variables to accumulate performance metrics
  cumulative_performance = {'Final Inventory': 0, 'Final Cash': 0, 'Total Trades': 0,'Profit': 0, 'Profit Percentage': 0,}

  for simulation in range(num_simulations):
    print(f"\nSimulating for {currency} - Run {simulation + 1}")
    market_maker = MarketMaker(initial_investment, max_inventory, trading_amount, trading_cost)
    market_maker.simulate_market(price_data)

    # Collect performance metrics for this simulation
    performance_metrics = market_maker.get_performance_metrics()

    # Accumulate performance metrics
    cumulative_performance['Final Inventory'] += performance_metrics['Final Inventory']
    cumulative_performance['Final Cash'] += performance_metrics['Final Cash']
    cumulative_performance['Total Trades'] += performance_metrics['Total Trades']
    cumulative_performance['Profit'] += performance_metrics['Profit']
    cumulative_performance['Profit Percentage'] += performance_metrics['Profit Percentage']


    # Calculate average performance metrics over all simulations
    average_performance = {key: value / num_simulations for key, value in cumulative_performance.items()}
    average_performance['Currency'] = currency

    # Append average performance metrics for this currency
    performance_results.append(average_performance)

# Convert performance results to DataFrame
performance_df = pd.DataFrame(performance_results)

# Display the average performance DataFrame
print(performance_df)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed



Simulating for EURUSD=X - Run 1

Simulating for EURUSD=X - Run 2

Simulating for EURUSD=X - Run 3

Simulating for EURUSD=X - Run 4

Simulating for EURUSD=X - Run 5

Simulating for EURUSD=X - Run 6

Simulating for EURUSD=X - Run 7

Simulating for EURUSD=X - Run 8

Simulating for EURUSD=X - Run 9

Simulating for EURUSD=X - Run 10

Simulating for EURUSD=X - Run 1

Simulating for EURUSD=X - Run 2

Simulating for EURUSD=X - Run 3

Simulating for EURUSD=X - Run 4

Simulating for EURUSD=X - Run 5

Simulating for EURUSD=X - Run 6

Simulating for EURUSD=X - Run 7

Simulating for EURUSD=X - Run 8

Simulating for EURUSD=X - Run 9

Simulating for EURUSD=X - Run 10

Simulating for GBPJPY=X - Run 1

Simulating for GBPJPY=X - Run 2

Simulating for GBPJPY=X - Run 3

Simulating for GBPJPY=X - Run 4

Simulating for GBPJPY=X - Run 5

Simulating for GBPJPY=X - Run 6

Simulating for GBPJPY=X - Run 7

Simulating for GBPJPY=X - Run 8

Simulating for GBPJPY=X - Run 9

Simulating for GBPJPY=X - Run 10

Simula

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed




Simulating for JPY=X - Run 9

Simulating for JPY=X - Run 10

Simulating for GBP=X - Run 1

Simulating for GBP=X - Run 2

Simulating for GBP=X - Run 3

Simulating for GBP=X - Run 4

Simulating for GBP=X - Run 5

Simulating for GBP=X - Run 6

Simulating for GBP=X - Run 7

Simulating for GBP=X - Run 8

Simulating for GBP=X - Run 9

Simulating for GBP=X - Run 10

Simulating for GBPAUD=X - Run 1

Simulating for GBPAUD=X - Run 2

Simulating for GBPAUD=X - Run 3

Simulating for GBPAUD=X - Run 4

Simulating for GBPAUD=X - Run 5

Simulating for GBPAUD=X - Run 6

Simulating for GBPAUD=X - Run 7

Simulating for GBPAUD=X - Run 8

Simulating for GBPAUD=X - Run 9

Simulating for GBPAUD=X - Run 10

Simulating for GBPCAD=X - Run 1

Simulating for GBPCAD=X - Run 2

Simulating for GBPCAD=X - Run 3

Simulating for GBPCAD=X - Run 4

Simulating for GBPCAD=X - Run 5

Simulating for GBPCAD=X - Run 6

Simulating for GBPCAD=X - Run 7

Simulating for GBPCAD=X - Run 8

Simulating for GBPCAD=X - Run 9

Simulati

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


...


Simulating for CNY=X - Run 1

Simulating for CNY=X - Run 2

Simulating for CNY=X - Run 3

Simulating for CNY=X - Run 4

Simulating for CNY=X - Run 5

Simulating for CNY=X - Run 6

Simulating for CNY=X - Run 7

Simulating for CNY=X - Run 8

Simulating for CNY=X - Run 9

Simulating for CNY=X - Run 10

Simulating for HKD=X - Run 1

Simulating for HKD=X - Run 2

Simulating for HKD=X - Run 3

Simulating for HKD=X - Run 4

Simulating for HKD=X - Run 5

Simulating for HKD=X - Run 6

Simulating for HKD=X - Run 7

Simulating for HKD=X - Run 8

Simulating for HKD=X - Run 9

Simulating for HKD=X - Run 10

Simulating for SGD=X - Run 1

Simulating for SGD=X - Run 2

Simulating for SGD=X - Run 3

Simulating for SGD=X - Run 4

Simulating for SGD=X - Run 5

Simulating for SGD=X - Run 6

Simulating for SGD=X - Run 7

Simulating for SGD=X - Run 8

Simulating for SGD=X - Run 9

Simulating for SGD=X - Run 10

Simulating for INR=X - Run 1

Simulating for INR=X - Run 2

Simulating for INR=X - Run 3

Simula

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed



Simulating for MXN=X - Run 1

Simulating for MXN=X - Run 2

Simulating for MXN=X - Run 3

Simulating for MXN=X - Run 4

Simulating for MXN=X - Run 5

Simulating for MXN=X - Run 6

Simulating for MXN=X - Run 7

Simulating for MXN=X - Run 8

Simulating for MXN=X - Run 9

Simulating for MXN=X - Run 10

Simulating for PHP=X - Run 1

Simulating for PHP=X - Run 2

Simulating for PHP=X - Run 3

Simulating for PHP=X - Run 4

Simulating for PHP=X - Run 5

Simulating for PHP=X - Run 6

Simulating for PHP=X - Run 7

Simulating for PHP=X - Run 8

Simulating for PHP=X - Run 9

Simulating for PHP=X - Run 10

Simulating for IDR=X - Run 1

Simulating for IDR=X - Run 2

Simulating for IDR=X - Run 3

Simulating for IDR=X - Run 4

Simulating for IDR=X - Run 5

Simulating for IDR=X - Run 6

Simulating for IDR=X - Run 7

Simulating for IDR=X - Run 8

Simulating for IDR=X - Run 9

Simulating for IDR=X - Run 10

Simulating for THB=X - Run 1

Simulating for THB=X - Run 2

Simulating for THB=X - Run 3

Simula

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed



Simulating for MYR=X - Run 1

Simulating for MYR=X - Run 2

Simulating for MYR=X - Run 3

Simulating for MYR=X - Run 4

Simulating for MYR=X - Run 5

Simulating for MYR=X - Run 6

Simulating for MYR=X - Run 7

Simulating for MYR=X - Run 8

Simulating for MYR=X - Run 9

Simulating for MYR=X - Run 10

Simulating for ZAR=X - Run 1

Simulating for ZAR=X - Run 2

Simulating for ZAR=X - Run 3

Simulating for ZAR=X - Run 4

Simulating for ZAR=X - Run 5

Simulating for ZAR=X - Run 6

Simulating for ZAR=X - Run 7

Simulating for ZAR=X - Run 8

Simulating for ZAR=X - Run 9

Simulating for ZAR=X - Run 10

Simulating for RUB=X - Run 1

Simulating for RUB=X - Run 2

Simulating for RUB=X - Run 3

Simulating for RUB=X - Run 4

Simulating for RUB=X - Run 5

Simulating for RUB=X - Run 6

Simulating for RUB=X - Run 7

Simulating for RUB=X - Run 8

Simulating for RUB=X - Run 9

Simulating for RUB=X - Run 10
     Final Inventory    Final Cash  Total Trades      Profit  \
0                0.0  1.000013e+0

In [115]:
# Return the entries with a profit percentage above 10%

# Filter the DataFrame to include only entries with Profit Percentage above 10%
profitable_currencies = performance_df[performance_df['Profit Percentage'] > 10]

# Display the filtered DataFrame
print(profitable_currencies)


     Final Inventory    Final Cash  Total Trades         Profit  \
254              0.0  6.107758e+05         260.0  108024.208225   
255              0.0  7.309744e+05         312.0  127677.333933   
256              0.0  8.529543e+05         364.0  149107.368307   
257              0.0  9.732091e+05         416.0  168816.576714   
258              0.0  1.096848e+06         468.0  191900.996215   
259              0.0  1.218748e+06         520.0  213251.428488   

     Profit Percentage Currency  
254          10.802421    IDR=X  
255          12.767733    IDR=X  
256          14.910737    IDR=X  
257          16.881658    IDR=X  
258          19.190100    IDR=X  
259          21.325143    IDR=X  
